<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/13_rnn/01_03_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networkによる電力予測


---
## 目的
Recurrent Neural Networkを使って電力予測を行う．ここで，今回はRecurrent Neural Networkの一種である，Long Short Term Memory（LSTM）を使用する．
また，PyTorchで使用されるデータセットオブジェクトの作成を行う．

## リカレントニューラルネットワーク
リカレントニューラルネットワークは，系列データを扱うことができるネットワークです． 
例えば，「今日は良い天気です」という文章において，「今日は」，「良い」という時系列データを与えると，次に現れる単語として「天気」を予測するという問題です． 
リカレントニューラルネットワークを利用することで，過去の系列情報から文脈の流れを考慮した予測ができるようになります． 
応用例として，30分後の電力を予測する，翌日の株価を予測するなどの予測モデル，音声認識や機械翻訳などがあります．

## リカレントニューラルネットワークの種類
リカレントニューラルネットワークにはいくつかの種類があります．

* Elman Network：一般的なリカレントニューラルネットワーク．１時刻前の情報を内部状態として，現時刻の入力と合わせて中間層に与える
<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/01_03_RNN/RNN.png?raw=true" width = 500>
* Jordan Network：１時刻前の出力層の情報を現時刻の入力と合わせて中間層に与える
* Echo state network (ESN)：一部の重みを乱数で初期化し，更新しない．中間層内のユニットは相互結合する
* Long Short-Term Memory (LSTM)：内部情報を記憶するメモリセルを持ち，複数のゲートによってメモリセルの情報を書き換えたり出力したりする
<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/01_03_RNN/LSTM.png?raw=true" width = 500>
* Gated Recurrent Unit (GRU)：内部情報の保持方法をLSTMよりもシンプルな構造にしたリカレントニューラルネットワーク 
<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/01_03_RNN/GRU.png?raw=true" width = 500>
* Bidirectional RNN：過去の情報だけでなく，未来の情報も利用する双方向のリカレントニューラルネットワーク

## リカレントニューラルネットワークの学習
リカレントニューラルネットワークは，時系列データを逐次与えます．
この流れを展開するとニューラルネットワークを時間方向につなげた大きなネットワークとみなすことができます．
そのため，リカレントニューラルネットワークの学習にもニューラルネットワークと同様に誤差逆伝播法を用いることができます．
リカレントニューラルネットワークでの誤差逆伝播法は， Back-propagation through time (BPTT)法と呼ばれています．

まず，図の黒矢印に従い，系列データを時刻$t=0$から順伝播します．
ネットワークは時刻ごとに別々にあるのではなく，１つのネットワークに対して逐次データを入力します．
その時，各時刻における各層の値は変わっていくので，それらを記憶しておきます．
また，順伝播時に各時刻における誤差を算出しておきます．

時刻$t=T$まで系列データの順伝播が終わると学習開始となります．
学習は誤差逆伝播法と同様に，BPTTでも誤差の勾配を求めて結合重みを更新します．
その際，時刻をさかのぼるように，時刻$t=T$の出力層から始めます．
学習では，以下の3箇所の結合重みを順番に更新します．
* 時刻tの出力層から時刻tの中間層間の結合重み
* 時刻tの中間層から時刻t-1の中間層間の結合重み
* 時刻tの中間層から時刻tの入力層間の結合重み

<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/01_03_RNN/back.png?raw=true" width = 500>

## モジュールのインポート
はじめに必要なモジュールをインポートする．


In [ ]:
from time import time
from os import path
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

### データのダウンロードと確認

プログラムの動作に必要なデータをダウンロードし，zipファイルを解凍する．

In [2]:
!wget -q http://www.mprg.cs.chubu.ac.jp/tutorial/ML_Lecture/SOLAR/data.zip
!unzip -q data.zip

データを確認してみます．最初の７つの値が曜日のone-hot vector，次の２４個の値は時間のone-hot vector，残りが電力，気温，湿度です．

In [ ]:
tmp_data = np.load("./data/train/BEMS_RNN_train_data.npy")
tmp_label = np.load("./data/train/BEMS_RNN_train_labels.npy")
print(tmp_data[0])
print(tmp_data.shape)
print(tmp_label[0])
print(tmp_label.shape)

## データセットオブジェクトの作成

電力データセットに対する，PyTorchのデータセットオブジェクト (`torch.utils.data.Dataset`) を作成します．
`Dataset`は，指定したデータセットを読み込み，学習やテストのためにデータを準備し生成するためのクラスです．
これまでの実習で使用したMNISTやCIFARデータセットはPyTorch (torchvision) 内に準備されているデータセットオブジェクトでした．
今回用いるデータセットは，torchvisonには存在しないため，自身で定義を行います．

まず，`__init__`関数により，必要なデータを読み込みます．
この時，`__init__`関数の引数を指定します．
`root`は読み込むデータセットを配置しているディレクトリ，`train`は学習またはテストデータのどちらを扱うかを指定する変数，`delay`は入力された情報の何時刻後を正解として用意するかを指定する変数，`time_window`は1サンプルあたり何時刻のデータを準備するかをしてする変数です．

まず，`root`および`train`変数から，学習またはテストデータを読み込みます．
その後，`delay`で指定した時刻を元に正解データを準備します．
最後に，`time_window`で指定した時間窓で1サンプルとなるように，データを作成し，`self.data`および`self.label`にデータを格納します．
これにより，`self.data`，`self.label`に入力データおよび正解データを格納します．

`__getitem__`関数で，指定したインデックス（`item`）のデータを取り出し，返します．

`__len__`関数は，このデータセットが保有するサンプル数を返すように定義を行います．

In [4]:
class BEMSDataset(torch.utils.data.Dataset):

    def __init__(self, root="./data", train=True, delay=1, time_window=10):
        super().__init__()
        self.root = root
        self.train = train
        self.delay = delay
        self.time_window = time_window

        # データの読み込み
        if self.train:
            data_src = np.load(path.join(self.root, 'train', 'BEMS_RNN_train_data.npy'))
            label_src = np.load(path.join(self.root, 'train', 'BEMS_RNN_train_labels.npy'))
        else:
            data_src  = np.load(path.join(self.root, 'test', 'BEMS_RNN_test_data.npy'))
            label_src = np.load(path.join(self.root, 'test', 'BEMS_RNN_test_labels.npy'))

        data_src = np.asarray(data_src[:-self.delay])
        label_src = np.asarray(label_src[self.delay:])

        self.data = []
        self.label = []
        for frame_i in range(len(data_src) - self.time_window):
            self.data.append(data_src[frame_i:frame_i+self.time_window])
            self.label.append(label_src[frame_i:frame_i+self.time_window])

        self.data = np.asarray(self.data)
        self.label = np.asarray(self.label)

    def __getitem__(self, item):
        d = self.data[item, :]
        l = self.label[item, :]
        return d, l

    def __len__(self):
        return self.data.shape[0]

## ネットワークモデルの定義

再帰型ニューラルネットワークを定義します．
ここでは，LSTM層1層，全結合層1層から構成されるネットワークとします．

LSTM層はRecurrent Neural Networkの一種です．
LSTMへの入力サイズはNoneとし，データにより変更できるようにしておきます．

`forward`関数では，定義した層を接続して処理するように記述します．
このとき，全結合層から出力された結果にくわえて，LSTMの隠れ状態とセル状態も同時に返し，次時刻への入力へと使用します．

また，LSTMをはじめとするRecurrent Neural Networkでは，内部に過去の入力情報から計算した値を保持しています．

In [14]:
class RNN(nn.Module):
    def __init__(self, n_hidden, rnn_name):
        super(RNN, self).__init__()
        self.rnn_name = rnn_name
        if rnn_name == "RNN":
          self.rnn = nn.RNNCell(34, n_hidden)
        elif rnn_name == "LSTM":
          self.rnn = nn.LSTMCell(34, n_hidden)
        elif rnn_name == "GRU":
          self.rnn = nn.GRUCell(34, n_hidden)
        self.l1 = nn.Linear(n_hidden, 1)
    
    def forward(self, x, hx, cx):
        if self.rnn_name == "RNN":
          hx = self.rnn(x, hx)
          h = self.l1(hx)
          return h, hx, cx
        elif self.rnn_name == "LSTM":
          hx, cx = self.rnn(x, (hx, cx))
          h = self.l1(hx)
          return h, hx, cx
        elif self.rnn_name == "GRU":
          hx = self.rnn(x, hx)
          h = self.l1(hx)
          return h, hx, cx

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．







In [15]:
n_hidden = 128
rnn_name = "LSTM"

model = RNN(n_hidden, rnn_name)
if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## 学習
先ほど定義したデータセットと作成したネットワークを用いて，学習を行います．

1回の誤差を算出するデータ数（ミニバッチサイズ）を100，学習エポック数を30とします．
また，1サンプルあたりのデータの長さ（time window）を10に指定します．

次にデータローダーを定義します．
データローダーでは，上で読み込んだデータセット（`train_data`）を用いて，for文で指定したミニバッチサイズでデータを読み込むオブジェクトを作成します．
この時，`shuffle=True`と設定することで，読み込むデータを毎回ランダムに指定します．

次に，誤差関数を設定します．
今回は，連続値を出力する回帰問題をあつかうため，`MSELoss`を`criterion`として定義します．

学習を開始します．

各更新において，学習用データと教師データをそれぞれ`data`と`label`とします．
まず，LSTMの隠れ状態とセル状態である`hx`と`cx`を`torch.zeros`を用いて初期化します．
この時，1次元目のサイズはバッチサイズに対応するように，`data`のサイズから自動的に決定します．

その後，学習モデルに`data`を与えて各クラスの確率yを取得します．
今回はLSTMを用いて時系列データを順次処理するため，for文を用いて，各時刻のデータを順番に入力し，結果を得ます．
そして，各クラスの確率yと教師ラベルtとの誤差を`criterion`で算出します．
また，認識精度も算出します．
そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 100
epoch_num = 10
time_window = 10

# データセットの読み込み・データローダーの設定
train_data = BEMSDataset(root="./data", train=True, delay=1, time_window=time_window)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定
criterion = nn.MSELoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()

start = time()
for epoch in range(1, epoch_num+1):
    total_loss = 0

    for data, label in train_loader:
        hx = torch.zeros(data.size()[1], n_hidden)
        cx = torch.zeros(data.size()[1], n_hidden)
        
        if use_cuda:
            data = data.cuda()
            label = label.cuda()
            hx = hx.cuda()
            cx = cx.cuda()
        
        for idx_window in range(time_window):
            y, hx, cx = model(data[idx_window], hx, cx)
            loss = criterion(y, label[idx_window])
            total_loss += loss.item()
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
  
    elapsed_time = time() - start
    print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch, total_loss, elapsed_time))

## テスト
学習したネットワークモデルを用いて評価（予測結果の可視化）を行います．
可視化にはmatplotlibを用います

In [ ]:
# データセットの読み込み・データローダーの設定
test_data = BEMSDataset(root="./data", train=False, delay=1, time_window=1)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

prediction_result = []
        
# 評価の実行
hx = torch.zeros(1, n_hidden)
cx = torch.zeros(1, n_hidden)
if use_cuda:
    hx = hx.cuda()
    cx = cx.cuda()
    
with torch.no_grad():
    for data, label in test_loader:
        
        if use_cuda:
            data = data.cuda()

        y, hx, cx = model(data[0], hx, cx)
        
        prediction_result.append(y.item())

prediction_result = np.array(prediction_result).flatten()


# 結果の表示
plt.figure()
plt.title(rnn_name)
plt.plot(test_data.label, color='red', label='true')
plt.plot(prediction_result.tolist(), color='blue', label='pred')
plt.legend()
plt.show()

## 課題

1. LSTM以外の再帰型ニューラルネットワークを用いた場合の結果を確認しましょう
    * `RNNCell`や`GRUCell`などがあります．
2. 電力予測について，入力データを現在の電力・気温・湿度のみ入力してみましょう